In [2]:
import numpy as np
import pandas as pd
import pickle
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import log_loss, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.datasets import fetch_20newsgroups_vectorized
from sklearn.feature_selection import chi2, f_classif
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import classification_report, accuracy_score,log_loss
from sklearn import metrics

/Users/hoangnguyen/miniconda3/envs/pydata/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/hoangnguyen/miniconda3/envs/pydata/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


build function to find optimal n_estimators
note: if learning_rate, reg_alpha, reg_lambda are not set
xgboost will use default values from https://github.com/dmlc/xgboost/blob/master/doc/parameter.md
e.g. eta = 0.3 instead of 0.1
if they are set, eta, alpha, lambda will be overwritten
the best n_estimators is Name + 1

In [3]:
# load data
with open("preprocessed_data.pkl", "rb") as f:
    train_data = pickle.load(f)
    train_label = pickle.load(f)
    label_list = pickle.load(f)

In [4]:
label_list

['Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer']

# Build model

# Multinomial regression

In [5]:
#fit a logistic regression model to the data
model = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, 
                           intercept_scaling=1, class_weight=None, random_state=None, 
                           solver='newton-cg', max_iter=100, multi_class='multinomial', 
                           verbose=0, warm_start=False, n_jobs=1)
model.fit(train_data, train_label)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

# Attributes of model

In [6]:
feat_labels = train_data.columns[0:]
feat_labels

Index(['Age', 'Size', 'HasName_0', 'HasName_1', 'TimeofDay_lateday',
       'TimeofDay_midday', 'TimeofDay_morning', 'TimeofDay_night', 'WeekDay_0',
       'WeekDay_1', 'WeekDay_2', 'WeekDay_3', 'WeekDay_4', 'WeekDay_5',
       'WeekDay_6', 'IsIntact_0.0', 'IsIntact_1.0', 'Sex_Female', 'Sex_Male',
       'MixorMultipleorSimple_Mix', 'MixorMultipleorSimple_Multiple',
       'MixorMultipleorSimple_Simple', 'Group_Herding', 'Group_Hound',
       'Group_NonSporting', 'Group_PitBull', 'Group_Sporting', 'Group_Terrier',
       'Group_Toy', 'Group_Unknown', 'Group_Working', 'IsMixColor_0',
       'IsMixColor_1'],
      dtype='object')

In [6]:
#for f in range(train_data.shape[1]):
#    print("{0}. Feature {1}: {2}".format(f + 1, feat_labels[indices[f]], importances[indices[f]]))

In [15]:
model.coef_

array([[-0.80591156, -0.07290339, -0.07325505,  0.07325491,  0.41675203,
        -0.15740431, -0.03639296, -0.22295489, -0.15728206, -0.22317624,
        -0.03505861, -0.19256136, -0.16552139,  0.48148046,  0.29211906,
         1.47220521, -1.47220534,  0.11498418, -0.11498432,  0.02941479,
         0.12069394, -0.15010887,  0.1471601 ,  0.27468738, -0.01123797,
        -0.5365071 ,  0.02678967,  0.20941728, -0.02026424,  0.08340706,
        -0.17345232, -0.02159708,  0.02159694],
       [ 0.13690816,  0.32291131,  0.63749166, -0.63749319, -0.59984404,
        -0.49389236,  0.92172918,  0.17200569,  0.1905389 ,  0.27326211,
        -0.55051448, -0.23153089,  0.02994081,  0.01737956,  0.27092247,
        -0.61725502,  0.61725349,  0.01288864, -0.01289017, -0.02613354,
         0.06004847, -0.03391647, -0.18353214, -0.36976383, -0.160007  ,
         0.10594142, -0.01684477, -0.09350974,  0.82549098, -0.21229967,
         0.10452321, -0.0129665 ,  0.01296497],
       [ 0.83547662, -0.0307

In [8]:
print(model.intercept_)

[ 1.1801698  -2.80256871 -0.72905616 -0.04801812  2.39947319]


In [19]:
import csv
a = [[-0.80591156, -0.07290339, -0.07325505,  0.07325491,  0.41675203,
        -0.15740431, -0.03639296, -0.22295489, -0.15728206, -0.22317624,
        -0.03505861, -0.19256136, -0.16552139,  0.48148046,  0.29211906,
         1.47220521, -1.47220534,  0.11498418, -0.11498432,  0.02941479,
         0.12069394, -0.15010887,  0.1471601 ,  0.27468738, -0.01123797,
        -0.5365071 ,  0.02678967,  0.20941728, -0.02026424,  0.08340706,
        -0.17345232, -0.02159708,  0.02159694],
       [ 0.13690816,  0.32291131,  0.63749166, -0.63749319, -0.59984404,
        -0.49389236,  0.92172918,  0.17200569,  0.1905389 ,  0.27326211,
        -0.55051448, -0.23153089,  0.02994081,  0.01737956,  0.27092247,
        -0.61725502,  0.61725349,  0.01288864, -0.01289017, -0.02613354,
         0.06004847, -0.03391647, -0.18353214, -0.36976383, -0.160007  ,
         0.10594142, -0.01684477, -0.09350974,  0.82549098, -0.21229967,
         0.10452321, -0.0129665 ,  0.01296497],
       [ 0.83547662, -0.03070819,  0.31773328, -0.31773125, -0.05091379,
         0.07397677,  1.13838044, -1.16144139,  0.11503093, -0.0270499 ,
         0.25958874,  0.29221808,  0.09547344, -0.38718638, -0.34807289,
        -0.61275828,  0.6127603 , -0.09791585,  0.09791787,  0.06641418,
         0.11916014, -0.1855723 , -0.00444256, -0.33530095, -0.04163943,
         0.84620782, -0.09868447, -0.24662367, -0.56504409,  0.29167955,
         0.15384983, -0.02422091,  0.02422294],
       [ 0.27842438, -0.07590826, -1.05450778,  1.05450879,  0.79087529,
         0.49844015, -1.07260977, -0.21670466, -0.08104193,  0.04899633,
         0.1994841 , -0.0242174 , -0.06037324,  0.00462205, -0.0874689 ,
         0.03975682, -0.0397558 , -0.04575457,  0.04575558,  0.03553327,
        -0.20676003,  0.17122778,  0.08572029,  0.15404077,  0.18468358,
        -0.0816835 ,  0.07726722,  0.11679378, -0.18762796, -0.38296006,
         0.03376689,  0.03592885, -0.03592784],
       [-0.4448976 , -0.14339146,  0.17253789, -0.17253926, -0.55686949,
         0.07887974, -0.95110689,  1.42909526, -0.06724584, -0.07203231,
         0.12650025,  0.15609157,  0.10048038, -0.1162957 , -0.12749973,
        -0.28194873,  0.28194735,  0.0157976 , -0.01579897, -0.10522871,
        -0.09314253,  0.19836987, -0.0449057 ,  0.27633662,  0.02820081,
        -0.33395864,  0.01147235,  0.01392235, -0.05255469,  0.22017312,
        -0.11868761,  0.02285564, -0.02285701]]



In [21]:
with open("output.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(a)

In [9]:
scores, pvalues1 = chi2(train_data, train_label)

In [10]:
pvalues1

array([  3.49615713e-251,   7.07987256e-005,   6.02755036e-237,
         6.54946873e-047,   2.06198523e-117,   2.08760280e-092,
         4.45233103e-104,   2.36717095e-067,   1.09655606e-001,
         7.69619931e-006,   8.14155073e-006,   2.40480444e-011,
         1.73908344e-008,   2.50025837e-027,   5.07208998e-027,
         1.68852984e-136,   0.00000000e+000,   3.41833525e-006,
         2.29523499e-005,   5.79325484e-002,   6.89261754e-023,
         3.44319814e-019,   5.29618856e-008,   2.75577366e-006,
         5.84668626e-004,   6.21572298e-074,   1.48109541e-002,
         5.69888263e-004,   6.68528488e-010,   2.11303626e-001,
         5.53686532e-002,   1.53403950e-002,   1.44439997e-001])

In [11]:
F, pvalues2 = f_classif(train_data, train_label)

In [12]:
pvalues2

array([  0.00000000e+000,   1.29837774e-008,   5.30535147e-298,
         5.30535147e-298,   1.55327625e-237,   4.38295741e-179,
         3.78077388e-109,   2.47511810e-069,   6.60571491e-002,
         8.97309755e-007,   1.09495027e-006,   3.81189835e-013,
         7.46971528e-010,   8.43544762e-033,   9.06647348e-033,
         0.00000000e+000,   0.00000000e+000,   9.49345483e-012,
         9.49345483e-012,   6.64717502e-007,   1.14208572e-028,
         8.32610293e-021,   1.26491842e-009,   7.53268293e-007,
         3.32705697e-004,   2.02052029e-087,   5.54695450e-003,
         2.48456942e-004,   1.01780982e-012,   2.07744457e-001,
         3.70711922e-002,   7.38876017e-004,   7.38876017e-004])